In [38]:
import pandas as pd
from pandas import DataFrame
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn import tree
from sklearn.metrics import (precision_score, recall_score, f1_score)
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [39]:
test_set = pd.read_csv('raw/TestSet.csv')
train_set = pd.read_csv('raw/TrainingSet.csv')
test_subset = pd.read_csv('raw/TestSubset.csv')
train_subset = pd.read_csv('raw/TrainingSubset.csv')

In [40]:
#将数据中无用的特征和该数据的标签提取出来
train=train_set.drop(['EbayID','QuantitySold','SellerName'],axis=1)
train_target=train_set['QuantitySold']

test = test_set.drop(['EbayID','QuantitySold','SellerName'],axis=1)
test_target = test_set['QuantitySold']

#将训练数据以及测试数据进行归一化
scaler = StandardScaler()
train = scaler.fit_transform(train)
test = scaler.fit_transform(test)

In [41]:
#使用CART对问题进行分类
clf=tree.DecisionTreeClassifier(criterion="gini",max_depth=6,min_samples_leaf=5)
clf=clf.fit(train,train_target)

In [23]:

test_pred = clf.predict(test) 

print("DecisionTreeClassifier training performance on testing dataset:" )
print("\tPrecision: %1.3f " % precision_score(test_target, test_pred))
print("\tRecall: %1.3f" % recall_score(test_target, test_pred))
print("\tF1: %1.3f \n" % f1_score(test_target, test_pred))
print("\tscore: %1.3f \n" % clf.score(test , test_pred))

DecisionTreeClassifier training performance on testing dataset:
	Precision: 0.826 
	Recall: 0.726
	F1: 0.773 

	score: 1.000 



In [47]:
#定义GridSearchCV中的参数；优化目标、遍历的参数及其取值；分类器
scores=['precision', 'recall']
parameters = {'criterion':('gini', 'entropy'), 'max_depth':[5,7,10],'min_samples_leaf':[1,2,5]}

#scoring = {'AUC': 'roc_auc'}
#处理数据不均匀问题
sample_weight = np.array([2.24 if i == 0 else 1 for i in train_target])
model=tree.DecisionTreeClassifier(sample_weight=sample_weight)
for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()  #打印括号
    
    clf=GridSearchCV(model,parameters,cv=5,scoring='%s_macro' % score,refit='AUC')
    clf.fit(train,train_target)
    
    #打印最好的参数
    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)  


    #
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()
    
    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    
    #打印在测试数据集中的预测结果。
    y_true, y_pred = test_target,clf.predict(test)
    print(classification_report(y_true, y_pred))
    print()


# Tuning hyper-parameters for precision
()
Best parameters set found on development set:
()
{'criterion': 'gini', 'max_depth': 10, 'min_samples_leaf': 2}
0.876 (+/-0.006) for {'criterion': 'gini', 'max_depth': 5, 'min_samples_leaf': 1}
0.876 (+/-0.006) for {'criterion': 'gini', 'max_depth': 5, 'min_samples_leaf': 2}
0.876 (+/-0.006) for {'criterion': 'gini', 'max_depth': 5, 'min_samples_leaf': 5}
0.878 (+/-0.008) for {'criterion': 'gini', 'max_depth': 7, 'min_samples_leaf': 1}
0.878 (+/-0.008) for {'criterion': 'gini', 'max_depth': 7, 'min_samples_leaf': 2}
0.878 (+/-0.008) for {'criterion': 'gini', 'max_depth': 7, 'min_samples_leaf': 5}
0.885 (+/-0.008) for {'criterion': 'gini', 'max_depth': 10, 'min_samples_leaf': 1}
0.885 (+/-0.008) for {'criterion': 'gini', 'max_depth': 10, 'min_samples_leaf': 2}
0.885 (+/-0.008) for {'criterion': 'gini', 'max_depth': 10, 'min_samples_leaf': 5}
0.876 (+/-0.012) for {'criterion': 'entropy', 'max_depth': 5, 'min_samples_leaf': 1}
0.876 (+/-0.012) for